# Scraping OpenReview

This is the scraping data file. provide the conferences to 'conference_invitations' list and run the file

In [1]:
!pip install openreview-py

In [2]:
import openreview
client = openreview.Client(baseurl='https://api.openreview.net')
# client = openreview.api.OpenReviewClient(baseurl='https://api2.openreview.net')

In [3]:
import pickle
from typing import Iterable, List
import pandas as pd

import itertools


def get_papers(conference_invitations: List[str]) -> Iterable['openreview.api.Note']:
    for invitation in conference_invitations:
        # this only gives us the first 1000 papers
        # paper_iterable = openreview.tools.efficient_iterget(
        #     client.get_notes,
        #     desc='Fetching papers',
        #     # kwargs for client.get_all_notes below
        #     invitation=invitation,
        #     details='directReplies'
        # )
        print(f'Fetching papers for invitation {invitation}')
        paper_iterable = client.get_all_notes(
            invitation=invitation,
            details='directReplies'
        )

        yield from paper_iterable


def get_paper_and_review_df(papers: List['openreview.api.Note']) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # note that to_json() method discards directReplies
    paper_df = pd.DataFrame([paper.to_json() for paper in papers])

    # we can join this with the paper_df on paper_df.id == review_df.forum
    flattened_reviews = itertools.chain.from_iterable(paper.details['directReplies'] for paper in papers)
    reply_df = pd.DataFrame(list(flattened_reviews))

    # when a reply is a review, it has 'Reviewer_' in ['signatures'], which is a list of strings
    review_df = reply_df[reply_df.signatures.apply(lambda x: any('Reviewer_' in s for s in x))]

    # other replies are replies that are not reviews, e.g. decision, meta-review
    other_replies_df = reply_df[~reply_df.index.isin(review_df.index)]

    return paper_df, review_df, other_replies_df

In [4]:
# scraping and saving to pickle file

conference_invitations = [
    'NeurIPS.cc/2022/Conference/-/Blind_Submission',
    'ICLR.cc/2023/Conference/-/Blind_Submission',
]


all_papers = list(get_papers(conference_invitations))

Fetching papers for invitation NeurIPS.cc/2022/Conference/-/Blind_Submission


Getting V1 Notes: 100%|█████████▉| 2821/2824 [00:03<00:00, 740.43it/s]


Fetching papers for invitation ICLR.cc/2023/Conference/-/Blind_Submission


Getting V1 Notes: 100%|█████████▉| 3792/3796 [00:06<00:00, 628.88it/s]


In [5]:

paper_df, review_df, other_replies_df = get_paper_and_review_df(all_papers)



In [6]:
# set types of date columns to datetime
for df in [paper_df, review_df, other_replies_df]:
    for col in df.columns:
        if "date" in col.lower():
            df[col] = pd.to_datetime(df[col], unit='ms')


In [7]:
with open('scraped-dataframes.pkl', 'wb') as f:
    content = {
        'paper_df': paper_df,
        'review_df': review_df,
        'other_replies_df': other_replies_df
    }
    pickle.dump(content, f)

## exploring what we have scraped

the rest of this notebook is a guide on how to use the scraped data

In [8]:
# loading from pickle file
with open('scraped-dataframes.pkl', 'rb') as f:
    data = pickle.load(f)
    paper_df = pd.DataFrame(data['paper_df'])
    review_df = pd.DataFrame(data['review_df'])
    other_replies_df = pd.DataFrame(data['other_replies_df'])
    


In [9]:
paper_df

,id,original,cdate,pdate,odate,mdate,tcdate,tmdate,ddate,number,content,forum,referent,invitation,replyto,readers,nonreaders,signatures,writers
0,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00.000000000,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,{'title': 'BEVFusion: A Simple and Robust LiDA...,zzDrPqn57DL,None,NeurIPS.cc/2022/Conference/-/Blind_Submission,None,[everyone],[],[NeurIPS.cc/2022/Conference],[NeurIPS.cc/2022/Conference]
1,zz0FC7qBpkh,7cZ94HqqMn,2022-05-16 21:50:30.798,2022-10-31 18:00:00.000000000,NaT,NaT,2022-05-16 21:50:30.798,2024-07-03 14:52:04.012,NaT,12079,{'title': 'The Missing Invariance Principle fo...,zz0FC7qBpkh,None,NeurIPS.cc/2022/Conference/-/Blind_Submission,None,[everyone],[],[NeurIPS.cc/2022/Conference],[NeurIPS.cc/2022/Conference]
2,zyrBT58h_J,GMUaJ0joiTU,2022-05-16 21:48:50.463,2022-10-31 18:00:00.000000000,NaT,NaT,2022-05-16 21:48:50.463,2024-07-03 14:54:30.521,NaT,9989,{'title': 'Sustainable Online Reinforcement Le...,zyrBT58h_J,None,NeurIPS.cc/2022/Conference/-/Blind_Submission,None,[everyone],[],[NeurIPS.cc/2022/Conference],[NeurIPS.cc/2022/Conference]
3,zvNMzjOizmn,r9JcSZTViEx,2022-05-16 21:49:03.370,2022-10-31 18:00:00.000000000,NaT,NaT,2022-05-16 21:49:03.370,2024-07-03 14:54:13.615,NaT,10262,{'title': 'Langevin Autoencoders for Learning ...,zvNMzjOizmn,None,NeurIPS.cc/2022/Conference/-/Blind_Submission,None,[everyone],[],[NeurIPS.cc/2022/Conference],[NeurIPS.cc/2022/Conference]
4,zuL5OYIBgcV,AH5xEKmMUnH,2022-05-16 21:42:24.613,2022-10-31 18:00:00.000000000,NaT,NaT,2022-05-16 21:42:24.613,2024-07-03 15:05:49.089,NaT,1891,"{'title': 'Non-deep Networks', 'authorids': ['...",zuL5OYIBgcV,None,NeurIPS.cc/2022/Conference/-/Blind_Submission,None,[everyone],[],[NeurIPS.cc/2022/Conference],[NeurIPS.cc/2022/Conference]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6615,-4Maz7s3YXz,SUZNkFXFIh,2022-09-22 12:35:25.224,2023-02-01 19:30:00.000000000,2022-09-29 16:15:00,NaT,2022-09-22 12:35:25.224,2023-02-13 23:28:19.214,NaT,2785,{'title': 'Towards Understanding Robust Memori...,-4Maz7s3YXz,None,ICLR.cc/2023/Conference/-/Blind_Submission,None,[everyone],[],[ICLR.cc/2023/Conference],[ICLR.cc/2023/Conference]
6616,-4DiyBMgv9m,SRt6LbSJc-u,2022-09-22 12:42:21.741,2023-02-01 19:30:00.000000000,2022-09-29 16:15:00,NaT,2022-09-22 12:42:21.741,2023-02-13 23:26:26.166,NaT,6214,{'title': 'Identifying Phase Transition Thresh...,-4DiyBMgv9m,None,ICLR.cc/2023/Conference/-/Blind_Submission,None,[everyone],[],[ICLR.cc/2023/Conference],[ICLR.cc/2023/Conference]
6617,-2zfgNS917,KQ3zOvPeO3,2022-09-22 12:30:15.845,2023-02-01 19:18:06.472999936,2022-09-29 16:15:00,NaT,2022-09-22 12:30:15.845,2024-11-25 09:16:48.563,NaT,143,{'title': 'BEVDistill: Cross-Modal BEV Distill...,-2zfgNS917,None,ICLR.cc/2023/Conference/-/Blind_Submission,None,[everyone],[],[ICLR.cc/2023/Conference],[ICLR.cc/2023/Conference]
6618,-1x2-lp1eZf,e-VvI0lh-OS,2022-09-22 12:35:28.070,2023-02-01 19:30:00.000000000,2022-09-29 16:15:00,NaT,2022-09-22 12:35:28.070,2023-02-13 23:28:18.359,NaT,2809,{'title': 'Rethinking Deep Spiking Neural Netw...,-1x2-lp1eZf,None,ICLR.cc/2023/Conference/-/Blind_Submission,None,[everyone],[],[ICLR.cc/2023/Conference],[ICLR.cc/2023/Conference]


In [10]:
review_df

,id,original,number,cdate,mdate,ddate,tcdate,tmdate,tddate,forum,replyto,invitation,content,signatures,readers,nonreaders,writers,pdate,odate
0,VrENUT1h6-,None,1,2022-07-04 14:14:33.004,NaT,NaT,2022-07-04 14:14:33.004,2022-07-04 14:14:33.004,NaT,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '7: Accept: Technically solid paper...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
1,B8g021IaaAm,None,2,2022-07-10 21:16:48.768,NaT,NaT,2022-07-10 21:16:48.768,2022-08-18 14:36:31.821,NaT,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '5: Borderline accept: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
2,HNAOCsCSzZw,None,3,2022-07-12 00:16:51.273,NaT,NaT,2022-07-12 00:16:51.273,2022-07-12 00:16:51.273,NaT,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '5: Borderline accept: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
3,uVn6Us0aP_,None,4,2022-07-12 00:54:51.965,NaT,NaT,2022-07-12 00:54:51.965,2022-07-12 00:54:51.965,NaT,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '4: Borderline reject: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
4,Hd6Ce57Ircd,None,5,2022-07-17 07:17:43.683,NaT,NaT,2022-07-17 07:17:43.683,2022-07-17 09:11:50.257,NaT,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,"{'rating': '6: Weak Accept: Technically solid,...",[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39365,nC5K4ES2guS,None,4,2022-11-01 07:09:25.916,NaT,NaT,2022-11-01 07:09:25.916,2022-11-01 07:09:25.916,NaT,-1x2-lp1eZf,-1x2-lp1eZf,ICLR.cc/2023/Conference/Paper2809/-/Official_R...,{'confidence': '4: You are confident in your a...,[ICLR.cc/2023/Conference/Paper2809/Reviewer_86ui],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT
39366,CA9y6TTZGq,None,5,2022-11-01 09:10:55.370,NaT,NaT,2022-11-01 09:10:55.370,2022-11-01 09:16:40.802,NaT,-1x2-lp1eZf,-1x2-lp1eZf,ICLR.cc/2023/Conference/Paper2809/-/Official_R...,{'confidence': '5: You are absolutely certain ...,[ICLR.cc/2023/Conference/Paper2809/Reviewer_BQAF],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT
39369,UgGANSuJ2pm,None,1,2022-10-17 08:38:33.803,NaT,NaT,2022-10-17 08:38:33.803,2022-10-17 15:06:08.781,NaT,-0tPmzgXS5,-0tPmzgXS5,ICLR.cc/2023/Conference/Paper211/-/Official_Re...,{'confidence': '4: You are confident in your a...,[ICLR.cc/2023/Conference/Paper211/Reviewer_Xyj5],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT
39370,eveHfO7L3IV,None,2,2022-10-24 22:13:52.568,NaT,NaT,2022-10-24 22:13:52.568,2022-11-22 09:42:15.900,NaT,-0tPmzgXS5,-0tPmzgXS5,ICLR.cc/2023/Conference/Paper211/-/Official_Re...,{'confidence': '4: You are confident in your a...,[ICLR.cc/2023/Conference/Paper211/Reviewer_6srd],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT


In [11]:
review_df.iloc[0].content

{'rating': '7: Accept: Technically solid paper, with high impact on at least one sub-area, or moderate-to-high impact on more than one areas, with good-to-excellent evaluation, resources, reproducibility, and no unaddressed ethical considerations.',
 'confidence': '3: You are fairly confident in your assessment. It is possible that you did not understand some parts of the submission or that you are unfamiliar with some pieces of related work. Math/other details were not carefully checked.',
 'summary': 'Towards the problem of current methods tend to fail at situations where hardware malfunctions, this paper presents a simple yet effective LiDAR-Camera fusion framework, namely BEVFusion. By disentangling camera pipeline from LiDAR network and using a dynamic fusion module, BEVFusion achieves SOTA performance and shows robustness against LiDAR or camera malfunction at the same time. An effective modification on the camera pipeline is also proposed to boost the final performance.',
 'stre

In [12]:
review_df.iloc[0].content

{'rating': '7: Accept: Technically solid paper, with high impact on at least one sub-area, or moderate-to-high impact on more than one areas, with good-to-excellent evaluation, resources, reproducibility, and no unaddressed ethical considerations.',
 'confidence': '3: You are fairly confident in your assessment. It is possible that you did not understand some parts of the submission or that you are unfamiliar with some pieces of related work. Math/other details were not carefully checked.',
 'summary': 'Towards the problem of current methods tend to fail at situations where hardware malfunctions, this paper presents a simple yet effective LiDAR-Camera fusion framework, namely BEVFusion. By disentangling camera pipeline from LiDAR network and using a dynamic fusion module, BEVFusion achieves SOTA performance and shows robustness against LiDAR or camera malfunction at the same time. An effective modification on the camera pipeline is also proposed to boost the final performance.',
 'stre

In [13]:
# this is how you can join them

merged_df = pd.merge(
    paper_df,
    review_df,
    left_on='id',
    right_on='forum',
    suffixes=('_paper', '_review')
)

merged_df


,id_paper,original_paper,cdate_paper,pdate_paper,odate_paper,mdate_paper,tcdate_paper,tmdate_paper,ddate_paper,number_paper,...,forum_review,replyto_review,invitation_review,content_review,signatures_review,readers_review,nonreaders_review,writers_review,pdate_review,odate_review
0,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '7: Accept: Technically solid paper...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
1,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '5: Borderline accept: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
2,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '5: Borderline accept: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
3,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '4: Borderline reject: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
4,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,"{'rating': '6: Weak Accept: Technically solid,...",[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24796,-1x2-lp1eZf,e-VvI0lh-OS,2022-09-22 12:35:28.070,2023-02-01 19:30:00,2022-09-29 16:15:00,NaT,2022-09-22 12:35:28.070,2023-02-13 23:28:18.359,NaT,2809,...,-1x2-lp1eZf,-1x2-lp1eZf,ICLR.cc/2023/Conference/Paper2809/-/Official_R...,{'confidence': '4: You are confident in your a...,[ICLR.cc/2023/Conference/Paper2809/Reviewer_86ui],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT
24797,-1x2-lp1eZf,e-VvI0lh-OS,2022-09-22 12:35:28.070,2023-02-01 19:30:00,2022-09-29 16:15:00,NaT,2022-09-22 12:35:28.070,2023-02-13 23:28:18.359,NaT,2809,...,-1x2-lp1eZf,-1x2-lp1eZf,ICLR.cc/2023/Conference/Paper2809/-/Official_R...,{'confidence': '5: You are absolutely certain ...,[ICLR.cc/2023/Conference/Paper2809/Reviewer_BQAF],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT
24798,-0tPmzgXS5,s3mA4mRGFzT,2022-09-22 12:30:24.402,2023-02-01 19:30:00,2022-09-29 16:15:00,NaT,2022-09-22 12:30:24.402,2023-02-13 23:29:46.530,NaT,211,...,-0tPmzgXS5,-0tPmzgXS5,ICLR.cc/2023/Conference/Paper211/-/Official_Re...,{'confidence': '4: You are confident in your a...,[ICLR.cc/2023/Conference/Paper211/Reviewer_Xyj5],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT
24799,-0tPmzgXS5,s3mA4mRGFzT,2022-09-22 12:30:24.402,2023-02-01 19:30:00,2022-09-29 16:15:00,NaT,2022-09-22 12:30:24.402,2023-02-13 23:29:46.530,NaT,211,...,-0tPmzgXS5,-0tPmzgXS5,ICLR.cc/2023/Conference/Paper211/-/Official_Re...,{'confidence': '4: You are confident in your a...,[ICLR.cc/2023/Conference/Paper211/Reviewer_6srd],[everyone],[],"[ICLR.cc/2023/Conference, ICLR.cc/2023/Confere...",NaT,NaT


In [14]:
# these are the reveiws for a paper
merged_df[merged_df['id_paper'] == paper_df.iloc[0].id]

,id_paper,original_paper,cdate_paper,pdate_paper,odate_paper,mdate_paper,tcdate_paper,tmdate_paper,ddate_paper,number_paper,...,forum_review,replyto_review,invitation_review,content_review,signatures_review,readers_review,nonreaders_review,writers_review,pdate_review,odate_review
0,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '7: Accept: Technically solid paper...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
1,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '5: Borderline accept: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
2,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '5: Borderline accept: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
3,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,{'rating': '4: Borderline reject: Technically ...,[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
4,zzDrPqn57DL,HXF7e04ViJI,2022-05-16 21:42:43.731,2022-10-31 18:00:00,NaT,NaT,2022-05-16 21:42:43.731,2024-07-03 15:05:11.771,NaT,2310,...,zzDrPqn57DL,zzDrPqn57DL,NeurIPS.cc/2022/Conference/Paper2310/-/Officia...,"{'rating': '6: Weak Accept: Technically solid,...",[NeurIPS.cc/2022/Conference/Paper2310/Reviewer...,[everyone],[],"[NeurIPS.cc/2022/Conference, NeurIPS.cc/2022/C...",NaT,NaT
